# Import data& libraries 

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys

In [9]:
X_train

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,...,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL
0,TRAIN_00000,COMPONENT3,1486,2011,7,200,0,3,93,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,154.0,75,0
1,TRAIN_00001,COMPONENT2,1350,2021,51,375,0,2,19,0,...,2.0,4.0,6.0,216.0,1454.0,0,NaN,44.0,652,0
2,TRAIN_00002,COMPONENT2,2415,2015,2,200,0,110,1,1,...,0.0,3.0,39.0,11261.0,41081.0,0,NaN,72.6,412,1
3,TRAIN_00003,COMPONENT3,7389,2010,2,200,0,8,3,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,133.3,7,0
4,TRAIN_00004,COMPONENT3,3954,2015,4,200,0,1,157,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,133.1,128,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,TRAIN_14090,COMPONENT3,1616,2014,8,200,0,2,201,1,...,NaN,NaN,NaN,NaN,NaN,0,NaN,135.4,16,0
14091,TRAIN_14091,COMPONENT1,2784,2013,2,200,0,3,85,0,...,NaN,NaN,NaN,NaN,NaN,0,14.5,117.5,1408,0
14092,TRAIN_14092,COMPONENT3,1788,2008,9,550,0,6,0,1,...,NaN,NaN,NaN,NaN,NaN,0,NaN,54.0,1301,0
14093,TRAIN_14093,COMPONENT2,2498,2009,19,550,0,2,4,0,...,7.0,8.0,100.0,1625.0,18890.0,0,NaN,44.3,652,0


In [8]:
path= "../../../Data/"

X_train = pd.read_csv(os.path.join(path, "train.csv"))
y = X_train["N_category"]
X_train = X_train[X_train.columns[:-1]]
X_train = X_train[X_train.columns[3:]]    #ID, 이미지 경로 제거
X_test = pd.read_csv(os.path.join(path, "test.csv"))
submission = pd.read_csv(os.path.join(path, "sample_submission.csv"))

KeyError: 'N_category'

In [ ]:
X_train.head(5)

In [ ]:
print(X_train.info())
print(f"\nSize of X_train: {len(X_train)}")

# Fill missing values

In [ ]:
X_train = X_train.fillna(X_train.mean())

# Process categorical values

In [ ]:
X_train["수술연월일"].head(5)

In [ ]:
X_train["수술연월일"]

In [ ]:
X_train["수술연월일"] = pd.to_datetime(X_train["수술연월일"])

In [ ]:
X_train["year"] = X_train["수술연월일"].dt.year
X_train["month"] = X_train["수술연월일"].dt.month
X_train["day"] = X_train["수술연월일"].dt.day
X_train["day_name"] = X_train["수술연월일"].dt.day_name

In [ ]:
X_train.info()

In [ ]:
date_list=[]
for i in range(len(X_train)):
    date_list.append(str(X_train["month"][i]).zfill(2) +str(X_train["day"][i]).zfill(2))

In [ ]:
X_train["수술연월일"] = date_list
X_train = X_train[X_train.columns[:-1]]

추후 위 코드 제거하고 요일 변수 전처리할것

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y, stratify=y, test_size=0.2)

# Train vanilla RF 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

In [ ]:
model.score(X_valid, y_valid)

In [ ]:
X_test

In [ ]:
def preprocess(X_train):
    X_train = X_train[X_train.columns[2:]]
    X_train = X_train.fillna(X_train.mean())    
    X_train["수술연월일"] = pd.to_datetime(X_train["수술연월일"])
    X_train["year"] = X_train["수술연월일"].dt.year
    X_train["month"] = X_train["수술연월일"].dt.month
    X_train["day"] = X_train["수술연월일"].dt.day
    X_train["day_name"] = X_train["수술연월일"].dt.day_name

    date_list=[]
    for i in range(len(X_train)):
        date_list.append(str(X_train["month"][i]).zfill(2) +str(X_train["day"][i]).zfill(2))

    X_train["수술연월일"] = date_list
    X_train = X_train[X_train.columns[:-1]]
    return X_train

In [ ]:
X_test = preprocess(X_test)

In [ ]:
pred = model.predict(X_test)

In [ ]:
submission["N_category"] = pred

In [ ]:
submission.to_csv("./submission_tabular.csv", index=False)